In [3]:
import numpy as np
import pandas as pd
import torch
import torchvision
import matplotlib.pyplot as plt
import os
import argparse
import matplotlib
from collections import OrderedDict
from datetime import datetime
from PIL import Image
from models_utils import * 
from data_utils import *
from tqdm import tqdm



In [4]:
class Args():
    def __init__(self,scenario='task',net='bnn',in_size=784,hidden_layers=[500,200],out_size=10,task_sequence=['KMNIST','FMNIST','MNIST'],
                lr=0.005,gamma=1,epochs_per_task=20,norm='bn',meta=[3],rnd_consolidation=False,ewc_lambda=0,ewc=False,si_lambda=0,si=False,bin_path=False,decay=1e-7,init='uniform',init_width=0.1,
                save=True,interleaved=False,beaker=False,fb=5e-3,n_bk=4,ratios=[1e-2,1e-3,1e-4,1e-5],areas=[1,2,4,8],
                device=0,seed=None,bit_num=3,upper_bound=1,noise_std=0.05,line_resistance=0.01,line_ratio=1):
        self.scenario=scenario
        self.net=net
        self.in_size=in_size
        self.hidden_layers=hidden_layers
        self.out_size=out_size
        self.task_sequence=task_sequence
        self.lr=lr
        self.gamma=gamma
        self.epochs_per_task=epochs_per_task
        self.norm=norm
        self.meta=meta
        self.rnd_consolidation=rnd_consolidation
        self.ewc_lambda=ewc_lambda
        self.ewc=ewc
        self.si_lambda=si_lambda
        self.si=si
        self.bin_path=bin_path
        self.decay=decay
        self.init=init
        self.init_width=init_width
        self.save=save
        self.interleaved=interleaved
        self.beaker=beaker
        self.fb=fb
        self.n_bk=n_bk
        self.ratios=ratios
        self.areas=areas
        self.device=device
        self.seed=seed
        self.bit_num=bit_num
        self.upper_bound=upper_bound
        self.noise_std=noise_std
        
        
def set_line_res_matrix(row_num, column_num, res_unit, ratio ):
    # res_unit: after normalization
    line_resistance_matrix=torch.zeros(row_num, column_num)
    for row in range(row_num):
        for column in range(column_num):
            line_resistance_matrix[row][column] = (row*ratio+ratio+2*(column+1)*(1/ratio))*res_unit  
    return line_resistance_matrix

def generate_failure_matrix(fail_num,len_1,len_2,previous_weight_matrix):
        weight_matrix = previous_weight_matrix
        
        count=0
        for try_num in range(1000000):
            if count > fail_num:
                break
            row = np.random.randint(0,len_1)
            column = np.random.randint(0,len_2)
            
            if int(weight_matrix[row][column]) == 5:
                count+=1
                weight_matrix[row][column]=np.random.randint(0,3)-1 
                
            else:
                pass
            
        
        #weight_matrix.add_(torch.normal(mean=0.0, std=0.1457, size=weight_matrix.shape))

        return weight_matrix

In [ ]:
#all the sequence： ['FMNIST','MNIST','KMNIST'],['FMNIST','KMNIST','MNIST'],['MNIST','FMNIST','KMNIST'],['MNIST','KMNIST','FMNIST'],['KMNIST','MNIST','FMNIST'] ['KMNIST','FMNIST','MNIST']

#for test_time in range(1):
    #for tasks_test in [['FMNIST','MNIST','KMNIST'],['KMNIST','FMNIST','MNIST']]:
#choose Bit=3 upper_bound=1.5 作为 探究 noise_std
#for bit_num_test in [1,2,3,4,5]:

#for line_resistance_test in [0.01,0.02,0.05,0.08,0.1,0.12,0.15,0.2,0.25,0.28]:
    #for line_ratio_test in [0.1,0.25,0.5,1,1.25,1.5,2,2.5,4,5]:
    
    
    
for epochs_per_task_test in [20]:
    
    
    fail_ratio=0.0001
    failure_weight_matrix_list={}

    for cycle_num in tqdm(range(epochs_per_task_test*3)):

        failure_weight_matrix_list[cycle_num]={}

        if cycle_num == 0:
            previous_matrix_0=torch.full((500,784),5)
            previous_matrix_1=torch.full((200,500),5)
            previous_matrix_2=torch.full((10,200),5)
            failure_weight_matrix_list[cycle_num][0]=generate_failure_matrix(fail_ratio*500*784,500,784,previous_matrix_0)
            failure_weight_matrix_list[cycle_num][1]=generate_failure_matrix(fail_ratio*200*500,200,500,previous_matrix_1)
            failure_weight_matrix_list[cycle_num][2]=generate_failure_matrix(fail_ratio*10*200,10,200,previous_matrix_2)

        else:
            previous_matrix_0=failure_weight_matrix_list[cycle_num-1][0]
            previous_matrix_1=failure_weight_matrix_list[cycle_num-1][1]
            previous_matrix_2=failure_weight_matrix_list[cycle_num-1][2]
            failure_weight_matrix_list[cycle_num][0]=generate_failure_matrix(fail_ratio*500*784,500,784,previous_matrix_0)
            failure_weight_matrix_list[cycle_num][1]=generate_failure_matrix(fail_ratio*200*500,200,500,previous_matrix_1)
            failure_weight_matrix_list[cycle_num][2]=generate_failure_matrix(fail_ratio*10*200,10,200,previous_matrix_2)

        #print(failure_weight_matrix_list[cycle_num][2][:1,:])

        #设置线阻矩阵
        line_resistance_test=0.02
        line_ratio_test=1
        line_resistance_matrix_0 = set_line_res_matrix(500,784,line_resistance_test/3652,line_ratio_test)
        line_resistance_matrix_1 = set_line_res_matrix(200,500,line_resistance_test/3652,line_ratio_test)
        line_resistance_matrix_2 = set_line_res_matrix(10,200,line_resistance_test/3652,line_ratio_test)

        line_res_matrix=[line_resistance_matrix_0,line_resistance_matrix_1,line_resistance_matrix_2]


        args=Args(meta=[3],bit_num=3,upper_bound=1.5,noise_std=0.05,epochs_per_task=epochs_per_task_test,line_resistance=line_resistance_test,line_ratio=line_ratio_test)
        device = torch.device("cuda:"+str(args.device) if torch.cuda.is_available() else "cpu")

        if args.seed is not None:
            torch.manual_seed(args.seed)
            np.random.seed(args.seed)

        date = datetime.now().strftime('%Y-%m-%d')
        time = datetime.now().strftime('%H-%M-%S')
        path = r'G:\Germany_project\Response_file\neural_network'
        if not(os.path.exists(path)):
            os.makedirs(path)

        createHyperparametersFile(path, args)

        train_loader_list = []
        test_loader_list = []
        dset_train_list = []
        task_names = []

        '''
        def shuffle(x):
            x=x.view(-1)
            permut = torch.from_numpy(np.random.permutation(784))
            a=[]
            for i in range(len(permut)):
                a.append(x[permut[i]])
            a=torch.tensor(a)
            a=a.view(1,28,28)
            return a
        #torchvision.transforms.Lambda(shuffle),
        '''



        for idx, task in enumerate(args.task_sequence):
            if task == 'MNIST':
                train_loader_list.append(mnist_train_loader)
                test_loader_list.append(mnist_test_loader)
                dset_train_list.append(mnist_dset_train)
                task_names.append(task)
            elif task == 'USPS':
                train_loader_list.append(usps_train_loader)
                test_loader_list.append(usps_test_loader)
                dset_train_list.append(usps_dset_train)
                task_names.append(task)
            elif task == 'CMNIST':
                train_loader_list.append(cmnist_train_loader)
                test_loader_list.append(cmnist_test_loader)
                dset_train_list.append(cmnist_dset_train)
                task_names.append(task)
            elif task == 'FMNIST':
                train_loader_list.append(fashion_mnist_train_loader)
                test_loader_list.append(fashion_mnist_test_loader)
                dset_train_list.append(fmnist_dset_train)
                task_names.append(task)

            elif task == 'KMNIST':
                train_loader_list.append(kmnist_train_loader)
                test_loader_list.append(kmnist_test_loader)
                dset_train_list.append(kmnist_dset_train)
                task_names.append(task)

            elif task == 'KaMNIST':
                train_loader_list.append(kamnist_train_loader)
                test_loader_list.append(kamnist_test_loader)
                dset_train_list.append(kamnist_dset_train)
                task_names.append(task)

            elif task == 'pMNIST':

                transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                           torchvision.transforms.Lambda(shuffle),
                              torchvision.transforms.Normalize(mean=(0.0,), std=(1.0,))])

                dset_train = torchvision.datasets.MNIST('./mnist_pytorch', train=True, transform=transform, target_transform=None, download=True)
                train_loader = torch.utils.data.DataLoader(dset_train, batch_size=100, shuffle=True,num_workers=0)

                dset_test = torchvision.datasets.MNIST('./mnist_pytorch', train=False, transform=transform, target_transform=None, download=True)
                test_loader = torch.utils.data.DataLoader(dset_test, batch_size=100, shuffle=False,num_workers=0)
                #train_loader, test_loader, dset_train = create_permuted_loaders('MNIST')
                train_loader_list.append(train_loader)
                test_loader_list.append(test_loader)
                dset_train_list.append(dset_train)
                task_names.append(task+str(idx+1))

            elif task == 'animals':
                animals_train_loader, animals_test_loader, animals_dset_train = process_cifar10(task)
                train_loader_list.append(animals_train_loader)
                test_loader_list.append(animals_test_loader)
                dset_train_list.append(animals_dset_train)
                task_names.append('animals')
            elif task == 'vehicles':
                vehicles_train_loader, vehicles_test_loader, vehicles_dset_train = process_cifar10(task)
                train_loader_list.append(vehicles_train_loader)
                test_loader_list.append(vehicles_test_loader)
                dset_train_list.append(vehicles_dset_train)
                task_names.append('vehicles')
            elif 'cifar100' in task:
                n_subset = int(task.split('-')[1])  # task = "cifar100-20" -> n_subset = 20
                train_loader_list, test_loader_list, dset_train_list = process_cifar100(n_subset)
                task_names = ['cifar100-'+str(i+1) for i in range(n_subset)]

        if args.interleaved:
            dset_train = torch.utils.data.ConcatDataset(dset_train_list)
            print(len(dset_train))
            train_loader = torch.utils.data.DataLoader(dset_train, batch_size=100, shuffle=True)
            train_loader_list = [train_loader]
        # Hyperparameters
        lr = args.lr
        epochs = args.epochs_per_task
        save_result = args.save
        #meta = args.meta
        ewc_lambda = args.ewc_lambda
        si_lambda = args.si_lambda
        archi = [args.in_size] + args.hidden_layers + [args.out_size]

        if args.net =='bnn':
            model = BNN( archi, init = args.init, width = args.init_width, norm = args.norm).to(device)
        elif args.net =='dnn':
            model = DNN( archi, init = args.init, width = args.init_width).to(device)
        elif args.net=='bcnn':
            model = ConvBNN(init = args.init, width = args.init_width, norm=args.norm).to(device)

        meta = {}
        for n, p in model.named_parameters():
            index = int(n[9])
            p.newname = 'l'+str(index)
            if ('fc' in n) or ('cv' in n):
                meta[p.newname] = args.meta[index-1] if len(args.meta)>1 else args.meta[0]



        print(model)
        #plot_parameters(model, path, save=save_result)

        previous_tasks_parameters = {}
        previous_tasks_fisher = {}

        # ewc parameters initialization
        if args.ewc:
            for n, p in model.named_parameters():
                if n.find('bn') == -1: #we dont store bn parameters as we allow task dependent bn
                    n = n.replace('.', '__')
                    previous_tasks_fisher[n] = []
                    previous_tasks_parameters[n] = [] 
        elif args.si:
            W = {}
            p_prev = {}
            p_old = {}
            omega = {}
            for n, p in model.named_parameters():
                if p.requires_grad:
                    n = n.replace('.', '__')
                    W[n] = p.data.clone().zero_()
                    omega[n] = p.data.clone().zero_()
                    if args.net=='bnn':
                        p_prev[n] = p.data.clone()  # or sign
                        if args.bin_path:
                            p_old[n] = p.data.sign().clone()
                        else:
                            p_old[n] = p.data.clone()
                    elif args.net=='dnn':
                        p_prev[n] = p.data.clone()
                        p_old[n] = p.data.clone()
        data = {}
        data['net'] = args.net
        data['scenario'] = args.scenario
        arch = ''
        if not(args.net=='bcnn'):
            for i in range(model.hidden_layers):
               arch = arch + '-' + str(model.layers_dims[i+1])

        data['arch'] = arch[1:]
        data['norm'] = args.norm
        data['lr'], data['meta'], data['ewc'], data['SI'], data['task_order'] = [], [], [], [], []  
        data['tsk'], data['epoch'], data['acc_tr'], data['loss_tr'] = [], [], [], []
        for i in range(len(test_loader_list)):
            data['acc_test_tsk_'+str(i+1)], data['loss_test_tsk_'+str(i+1)] = [], []

        name = '_'+data['net']+'_'+data['arch']+'_'
        for t in range(len(task_names)):
            if ('cifar100' in task_names[t]) and ('cifar100' in name):
                pass
            else:
                name = name+task_names[t]+'-'

        bn_states=[]        
        lrs = [lr*(args.gamma**(-i)) for i in range(len(train_loader_list))] 

        if args.beaker:
            optimizer = Adam_bk(model.parameters(), lr = lr, n_bk=args.n_bk, ratios=args.ratios, areas=args.areas, feedback=args.fb, meta=meta, weight_decay=args.decay, path=path)
        if args.si:
            optimizer = torch.optim.Adam(model.parameters(), lr = lr, weight_decay = args.decay)

        for task_idx, task in enumerate(train_loader_list):
            if not(args.beaker or args.si):
                optimizer = Adam_meta(model.parameters(), lr = lrs[task_idx], meta = meta, weight_decay = args.decay)



        #进行计算的过程
        for epoch in tqdm(range(1, epochs+1)):

            print('No.epoch:{}'.format((task_idx)*epochs+epoch))

            if args.ewc:
                train(model, task, task_idx, optimizer, device, args, prev_cons=previous_tasks_fisher, 
                        prev_params=previous_tasks_parameters) 
            elif args.si:
                train(model, task, task_idx, optimizer, device, args, prev_cons=omega, path_integ=W, prev_params=(p_prev, p_old) ) 
            else:
                train(model, task, task_idx, optimizer, device, args)

            data['task_order'].append(task_idx+1)
            data['tsk'].append(task_names[task_idx])
            data['epoch'].append(epoch)
            data['lr'].append(optimizer.param_groups[0]['lr'])

            train_accuracy, train_loss = test(model, task, device, verbose=True, line_res_matrix=line_res_matrix,fail_matrix=failure_weight_matrix_list[(task_idx)*epochs+epoch-1])

            data['acc_tr'].append(train_accuracy)
            data['loss_tr'].append(train_loss)
            data['meta'].append(meta)
            data['ewc'].append(ewc_lambda)
            data['SI'].append(si_lambda)

            current_bn_state = model.save_bn_states()

            for other_task_idx, other_task in enumerate(test_loader_list):

                if args.scenario == 'task':
                    if other_task_idx>=task_idx:
                        model.load_bn_states(current_bn_state)
                        test_accuracy, test_loss = test(model , other_task, device, verbose=(other_task_idx==task_idx),line_res_matrix=line_res_matrix,
                                                        fail_matrix=failure_weight_matrix_list[(task_idx)*epochs+epoch-1])
                    else:
                        model.load_bn_states(bn_states[other_task_idx])
                        test_accuracy, test_loss = test(model , other_task, device,line_res_matrix=line_res_matrix, 
                                                        fail_matrix=failure_weight_matrix_list[(task_idx)*epochs+epoch-1])

                elif args.scenario =='domain':
                    test_accuracy, test_loss = test(model, other_task, device, verbose=True,line_res_matrix=line_res_matrix, 
                                                    fail_matrix=failure_weight_matrix_list[(task_idx)*epochs+epoch-1])

                data['acc_test_tsk_'+str(other_task_idx+1)].append(test_accuracy)
                print(data['acc_test_tsk_'+str(other_task_idx+1)])
                data['loss_test_tsk_'+str(other_task_idx+1)].append(test_loss)

            model.load_bn_states(current_bn_state)

        plot_parameters(model, path, save=save_result)
        # Uncomment for hidden weight histogram of Fig. 2g,h
        #time = datetime.now().strftime('%H-%M-%S')
        #for l in range(model.hidden_layers + 1):
        #    torch.save(model.layers['fc'+str(l+1)].weight.org.data, path+'/'+time+'_weights_fc'+str(l+1)+'.pt')

        #将已经test过的内容的BN参数进行保留
        bn_states.append(current_bn_state)

        #高级对比内容
        if args.ewc:
            fisher = estimate_fisher(model, dset_train_list[task_idx], device, num=5000, empirical=True)
            for n, p in model.named_parameters():
                if n.find('bn') == -1: # not batchnorm
                    n = n.replace('.', '__')

                    # random consolidation
                    if args.rnd_consolidation:
                        idx = torch.randperm(fisher[n].nelement())
                        previous_tasks_fisher[n].append(fisher[n].view(-1)[idx].view(fisher[n].size()))

                    # EWC consolidation, comment when using random consolidation
                    previous_tasks_fisher[n].append(fisher[n])
                    previous_tasks_parameters[n].append(p.detach().clone())

        elif args.si:
            omega = update_omega(model, omega, p_prev, W)
            for n, p in model.named_parameters():
                if n.find('bn') == -1: # not batchnorm
                    n = n.replace('.','__')
                    if args.net=='bnn':
                        p_prev[n] = p.org.detach().clone()  # or sign
                    else:
                        p_prev[n] = p.detach().clone()



        i=0

        for (n, p) in model.named_parameters():

            if (n.find('bias') == -1) and (len(p.size()) != 1):  #bias or batchnorm weight -> no plot
                if model.__class__.__name__.find('B') != -1:  #BVGG -> plot p.org
                    if hasattr(p,'org'):
                        weights_org_hist = p.org.data.flatten().cpu().numpy()
                        weights_org = p.org.data.cpu().numpy()
                        weights_hist = p.data.flatten().cpu().numpy()
                        weights = p.data.cpu().numpy()

                        np.savetxt(path+'//'+args.task_sequence[task_idx]+'_FC_'+str(i)+'_weights_org_hist.csv',weights_org_hist,delimiter=',')
                        np.savetxt(path+'//'+args.task_sequence[task_idx]+'_FC_'+str(i)+'_weights_org.csv',weights_org,delimiter=',')
                        np.savetxt(path+'//'+args.task_sequence[task_idx]+'_FC_'+str(i)+'_weights_hist.csv',weights_hist,delimiter=',')
                        np.savetxt(path+'//'+args.task_sequence[task_idx]+'_FC_'+str(i)+'_weights.csv',weights,delimiter=',')
                        i+=1
                    else:
                        pass
                else:
                    weight_data= p.data.cpu().numpy()
                    np.savetxt(path+'//'+'FC_'+str(i)+'_weights.csv',weight_data,delimiter=',')
                    i+=1
            else:
                pass





        time = datetime.now().strftime('%H-%M-%S')
        df_data = pd.DataFrame(data)
        if save_result:
            df_data.to_csv(path +'/'+time+name+'.csv', index = False)

        data_write={}
        for i_task in range(len(args.task_sequence)):
            data_write[args.task_sequence[i_task]]=data['acc_test_tsk_'+str(i_task+1)]
        d=pd.DataFrame(data_write)

        d.to_csv(path+'/data.csv')




